In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
#quora question pairs-kaggle
df = pd.read_csv("/home/uv/Downloads/train.csv")

In [3]:
df = df.sample(1000)

In [4]:
df.shape

(1000, 6)

In [5]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
393096,393096,525855,11078,What are the character traits of a Chihuahua/P...,What are the character traits of a Pitbull/Chi...,1
329811,329811,16025,19619,Is the United States a true democracy or a rep...,Is the United States truly a democracy?,1
56929,56929,78733,97964,What is the procedure and documents required f...,What are the documents required for Indian pas...,1
251099,251099,82430,365131,What are some ways to make gold food coloring?,How can you make blue food coloring?,0
342869,342869,124919,470908,My Speech teacher told me to stand up straight...,I have been out of high school for four years....,0


In [6]:
#Normal WOrd tokenizer nltk and moses tokenizer
def tokenize_s(s,moses_tok):
        from nltk.tokenize import word_tokenize
        if moses_tok:
            s = ' '.join(word_tokenize(s))
            s = s.replace(" n't ", "n 't ")  # HACK to get ~MOSES tokenization
            return s.split()
        else:
            return word_tokenize(s)

In [7]:
def build_vocab_text(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [9]:
sentences_m = df["question1"].progress_apply(lambda x: tokenize_s(x,True)).values
vocab_m = build_vocab_text(sentences_m)
print({k: vocab_m[k] for k in list(vocab_m)[:5]})

100%|██████████| 1000/1000 [00:00<00:00, 197332.58it/s]

{'What': 364, 'are': 174, 'the': 463, 'character': 1, 'traits': 1}


In [10]:
sentences = df["question1"].progress_apply(lambda x: x.split()).values
vocab = build_vocab_text(sentences)
print({k: vocab[k] for k in list(vocab)[:5]})

100%|██████████| 1000/1000 [00:00<00:00, 283936.10it/s]

{'What': 353, 'are': 174, 'the': 463, 'character': 1, 'traits': 1}


In [11]:
vocab_m == vocab

False

In [33]:
#vocab_m.keys()

In [15]:
shared_items = {k: vocab[k] for k in vocab if k in vocab_m and vocab[k] == vocab_m[k]}
print(len(shared_items))

2170


In [34]:
#vocab.keys() -vocab_m.keys()

In [ ]:
vocab_m.keys()

# Set path

In [17]:
bos = '<p>'
eos = '</p>'

In [18]:
def set_w2v_path(w2v_path):
        w2v_path = w2v_path

In [20]:
w2v_path = '/home/uv/Documents/document_embedding/fastText/crawl-300d-2M.vec'

In [21]:
 def get_word_dict(sentences, tokenize=True):
        # create vocab of words
        word_dict = {}
        sentences = [s.split() if not tokenize else tokenize_s(s,True) for s in sentences]
        print("sentences after tokenization",sentences)
        for sent in sentences:
            for word in sent:
                if word not in word_dict:
                    word_dict[word] = ''
        word_dict[bos] = ''
        word_dict[eos] = ''
        return word_dict

In [22]:
def build_vocab(sentences, tokenize=True):
        #assert hasattr(self, 'w2v_path'), 'w2v path not set'
        word_dict = get_word_dict(sentences, tokenize)
        word_vec = get_w2v(word_dict)
        print('Vocab size : %s' % (len(word_vec)))
        #print(self.word_vec)
        return word_dict,word_vec

In [23]:
 def get_w2v(word_dict):
        #assert hasattr(self, 'w2v_path'), 'w2v path not set'
        # create word_vec with w2v vectors
        word_vec = {}
        with open(w2v_path, encoding='utf-8') as f:
            for line in f:
                word, vec = line.split(' ', 1)
                if word in word_dict:
                    word_vec[word] = np.fromstring(vec, sep=' ')
        print('Found %s(/%s) words with w2v vectors' % (len(word_vec), len(word_dict)))
        print("word_vec",word_vec.keys())
        return word_vec

## Load fastext

In [24]:
def get_w2v(w2v_path,K):
        #assert hasattr(self, 'w2v_path'), 'w2v path not set'
        # create word_vec with w2v vectors
        k=0
        word_vec = {}
        with open(w2v_path, encoding='utf-8') as f:
            for line in f:
                word, vec = line.split(' ', 1)
                if k <= K:
                    word_vec[word] = np.fromstring(vec, sep=' ')
                    k += 1
                if k > K:
                    if word in [bos, eos]:
                        word_vec[word] = np.fromstring(vec, sep=' ')

                if k > K and all([w in word_vec for w in [bos, eos]]):
                    break
        print('Found %s words with w2v vectors' % (len(word_vec)))
        return word_vec

In [25]:
fastext_vec = get_w2v(w2v_path,K=10000)

Found 10003 words with w2v vectors


In [32]:
#fastext_vec.keys()

In [27]:
import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [28]:
oov = check_coverage(vocab_m,fastext_vec)

100%|██████████| 3147/3147 [00:00<00:00, 1312727.47it/s]

Found embeddings for 67.78% of vocab
Found embeddings for  89.97% of all text


In [31]:
#oov

In [30]:
'?' in fastext_vec

True

# References

https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings
https://www.kaggle.com/theoviel/improve-your-score-with-some-text-preprocessing